In [36]:
import scrapy
import chromadb
import requests
from bs4 import SoupStrainer, BeautifulSoup
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv


load_dotenv()

import getpass
import os

os.environ["OPENAI_API_KEY"]

bs4_strainer = SoupStrainer(
    ["p", "h1", "h2", "h3", "h4", "h5", "h6"]
) 
loader = WebBaseLoader(
    web_paths=(
        "https://en.wikipedia.org/wiki/Six_(musical)",
        "https://en.wikipedia.org/wiki/Catherine_of_Aragon",
        "https://en.wikipedia.org/wiki/Anne_Boleyn",
        "https://en.wikipedia.org/wiki/Jane_Seymour ",
        "https://en.wikipedia.org/wiki/Anne_of_Cleves",
        "https://en.wikipedia.org/wiki/Catherine_Howard",
        "https://en.wikipedia.org/wiki/Catherine_Parr",
        "https://en.wikipedia.org/wiki/Henry_VIII",
    ),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()


print(len(docs[7].page_content))
# Optionally print the text content to verify
print(docs[7].page_content)

ImportError: cannot import name '_parse_google_docstring' from 'langchain_core.utils.function_calling' (/opt/anaconda3/envs/chatbotenv/lib/python3.12/site-packages/langchain_core/utils/function_calling.py)

In [32]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
documents = text_splitter.split_documents(docs)

print(len(documents))
print(len(documents[4].page_content))
print(documents[4].page_content)

vectorstore = Chroma.from_documents(documents=documents, embedding=OpenAIEmbeddings(model="text-embedding-ada-002"))

398
294
exposes flaws in the others' claims to winning. Katherine recounts her romantic history, having had many suitors even as a child, and at first relishes her attractiveness before later revealing the emotional trauma and sexual abuse she faced in each of these relationships ("All You Wanna Do").


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("Who was Anne Boleyn?")
print(len(retrieved_docs))
print(retrieved_docs[0].page_content)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")